In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import youtubecollector as ytc
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [4]:
youtube_client = ytc.youtube_client.create_youtube_client("/home/dim/Documents/projecten/extremisme/youtube/DataCollection/api.conf")

Google Developer Api key: ········


In [8]:
video_file = "videos_for_graphic.csv"

In [29]:
import csv as _csv
from collections import namedtuple as _namedtuple

import os as _os


def is_empty_file(filename: str):
    if not isinstance(filename, str):
        raise Exception(f"filename should be a string of an existing file")

    if not _os.path.exists(filename):
        raise Exception(f"{filename} doesn't exists")

    return _os.stat(filename).st_size == 0


def _convert_to_dictionary(obj):
    return {field: getattr(obj, field) for field in obj._fields}


video = _namedtuple('video', ('video_id',
                              'video_published',
                              'channel_id',
                              'video_title',
                              'video_description',
                              'video_channel_title',
                              'video_tags',
                              'video_category_id',
                              'video_default_language',
                              'video_duration',
                              'video_view_count',
                              'video_comment_count',
                              'video_likes_count',
                              'video_dislikes_count',
                              'video_topic_ids',
                              'video_topic_categories'
                              ))

def _get_video_header():
    return video._fields

def _get_video_metadata(video_id, youtube_client):
    return youtube_client.videos().list(
        part='snippet,contentDetails,statistics,topicDetails',
        id=video_id
    ).execute()

def _get_topic_ids(metadata):
    if "topicDetails" in metadata:
        return metadata['topicDetails'].get('relevantTopicIds', "not set")
    else:
        return "not set"


def _get_topic_categories(metadata):
    if "topicDetails" in metadata:
        return metadata['topicDetails'].get('topicCategories', "not set")
    else:
        return "not set"


def convert_to_videos(response, youtube_client):
    videos = list()
    for data in response['items']:
        video_id = data['id']
        video_metadata = _get_video_metadata(video_id, youtube_client)
        metadata = video_metadata['items'][0]

        next_video = video(video_id=video_id,
                           video_published=data['snippet']['publishedAt'],
                           channel_id=data['snippet']['channelId'],
                           video_title=data['snippet']['title'],
                           video_description=data['snippet'].get('description', 'not set'),
                           video_channel_title=data['snippet']['channelTitle'],
                           video_tags=metadata['snippet'].get('tags', 'not set'),
                           video_category_id=metadata['snippet'].get('categoryId', 'not set'),
                           video_default_language=metadata['snippet'].get('defaultLanguage', 'not set'),
                           video_duration=metadata['contentDetails']['duration'],
                           video_view_count=metadata['statistics'].get('viewCount', 0),
                           video_comment_count=metadata['statistics'].get('commentCount', 0),
                           video_likes_count=metadata['statistics'].get('likeCount', 0),
                           video_dislikes_count=metadata['statistics'].get('dislikeCount', 0),
                           video_topic_ids=_get_topic_ids(metadata),
                           video_topic_categories=_get_topic_categories(metadata)
                           )
        videos.append(next_video)

    return videos


def write_videos(videos, video_file):
    with open(video_file, "a") as csv_file:
        writer = _csv.DictWriter(csv_file, fieldnames=_get_video_header())

        for video_row in videos:
            writer.writerow(_convert_to_dictionary(video_row))

In [30]:
for vid in vids:
    meta = _get_video_metadata(vid, youtube_client)
    #print(meta)
    meta2 = convert_to_videos(meta, youtube_client)
    write_videos(meta2, video_file)